# HIV Kinda Sucks
By: Michael Remley

Patent Pending

In [2]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [3]:
init = State(R=???,L=???,E=???,V=???)
def update_func(system,state):
    dRdt = system.Γ * system.τ - system.μ * state.R - system.β * state.R * state.V
    dLdt = system.ρ * system.β * state.R * state.V - system.μ * state.L - system.α * state.L
    dEdt = (1 - system.ρ) * system.β * state.R * state.V - system.α * state.L - system.δ * state.E
    dVdt = system.π * state.E - system.σ * state.V
    
    r += dRdt
    l += dLdt
    e += dEdt
    v += dVdt
    
    return state(R=r,L=l,E=e,V=v)

system = System(β=0.00027,
                μ=1.36E-3,
                Γ=1.36,
                τ=0.2,
                ρ=0.1,
                δ=0.33,
                α=3.6E-2,
                σ=2,
                π=100,
                init=init)

SyntaxError: invalid syntax (<ipython-input-3-00548a99fb61>, line 1)